In [2]:
import theano
import theano.tensor as T
import lasagne

In [ ]:
lasagne.updates.nesterov_momentum

# Load pre-trained weights

In [2]:
import os, glob
from natsort import natsorted
import numpy as np

root_dir = "/home/hoa/Desktop/multiLabel/"
os.chdir(root_dir+'pretrained/parameters_releasing/')

In [8]:
Wnames = natsorted([w[:-4] for _,w in enumerate(glob.iglob('W*'))])

Ws = {}
for W in Wnames:
    Ws[W] = np.load(W+".npy")
    print W, Ws[W].shape
    
varName = natsorted([b[:-7] for _,b in enumerate(glob.iglob('b*'))])

bs = {}
for b in varName:
    bs[b] = np.load(b+"_65.npy")
    print b, bs[b].shape

W0_1 (128, 48, 5, 5)
W0_3 (192, 192, 3, 3)
W0_4 (128, 192, 3, 3)
W1_1 (128, 48, 5, 5)
W1_3 (192, 192, 3, 3)
W1_4 (128, 192, 3, 3)
W_0 (96, 3, 11, 11)
W_2 (384, 256, 3, 3)
W_5 (9216, 4096)
W_6 (4096, 4096)
W_7 (4096, 1000)
b0_1 (128,)
b0_3 (192,)
b0_4 (128,)
b1_1 (128,)
b1_3 (192,)
b1_4 (128,)
b_0 (96,)
b_2 (384,)
b_5 (4096,)
b_6 (4096,)
b_7 (1000,)


"""Convert params to from c01b to bc01"""
def dimConv(W):
    assert len(W.shape)==4, 'not 4D tensor'
    c,row,col,b = W.shape
    out = np.ndarray((b,c,row,col), dtype=W.dtype)
    for i in xrange(b):
        out[i,:,:,:]=W[:,:,:,i]
    
    return out

for key in natsorted(Ws.keys()):
    if len(Ws[key].shape)==4:
        Ws[key]=dimConv(Ws[key])
    print key, Ws[key].shape
    np.save(key+".npy", Ws[key])



In [9]:
for key in Ws.keys():
    print key, Ws[key].dtype

W0_4 float32
W0_1 float32
W0_3 float32
W_5 float32
W1_4 float32
W_7 float32
W_6 float32
W1_1 float32
W_0 float32
W1_3 float32
W_2 float32


# Data

### Load dataset

In [58]:
# Saving the objects
with open("Data/MSRCv2/lables.pkl",'w') as f:
    pickle.dump([labelNames, labelIDs], f)

# Getting back the object
with open("Data/MSRCv2/lables.pkl",'r') as f:
    test1, test2 = pickle.load(f)

If you have a lot of data, you can reduce the file size by passing `protocol=-1` to `dump()`; `pickle` will then use the best available protocol instead of the default historical (and more backward-compatible) protocol. In this case, the file must be opened in binary mode (`wb` and `rb`, respectively).

In [66]:
print test1; test2

['aeroplane' 'bicycle' 'bird' 'boat' 'body' 'book' 'building' 'car' 'cat'
 'chair' 'cow' 'dog' 'face' 'flower' 'grass' 'mountain' 'road' 'sheep'
 'sign' 'sky' 'tree' 'water']


{'aeroplane': 0,
 'bicycle': 1,
 'bird': 2,
 'boat': 3,
 'body': 4,
 'book': 5,
 'building': 6,
 'car': 7,
 'cat': 8,
 'chair': 9,
 'cow': 10,
 'dog': 11,
 'face': 12,
 'flower': 13,
 'grass': 14,
 'mountain': 15,
 'road': 16,
 'sheep': 17,
 'sign': 18,
 'sky': 19,
 'tree': 20,
 'water': 21}

In [27]:
np.save("Data/MSRCv2/Xtr.npy", Xtr)
np.save("Data/MSRCv2/Xva.npy", Xva)
np.save("Data/MSRCv2/Ytr.npy", Ytr)
np.save("Data/MSRCv2/Yva.npy", Yva)

In [51]:
def load_dataset(path="Data/MSRCv2/"):
    dataset={}
    setNames = [name[-7:-4] for _,name in enumerate(glob.iglob(path+"*.npy"))]
    for name in setNames:
        dataset[name]=np.load(path+name+".npy")
        print name, dataset[name].shape
    return dataset

dataset = load_dataset()

Xtr (441, 256, 256, 3)
Xva (147, 256, 256, 3)
Ytr (441, 22)
Yva (147, 22)


### Data augmentation

In [12]:
import cv2
def data_aug(mat, isinput=True, n_ops=10):
    # resize, crop, mirror if "input", simply repeat if "target"
    n_samples = mat.shape[0]
    
    if isinput:
        out = np.empty((n_samples*n_ops, 227, 227, 3), dtype="uint8")        
        for i in xrange(n_samples):
            # original images resized to 227x227
            out[i*n_ops:(i+1)*n_ops,:,:,:] = cv2.resize(mat[i,:,:,:],
                                                 (227,227))
            # 4 crops
            out[i*n_ops+1:((i+1)*n_ops+1),:,:,:] = mat[i,:227,:227,:]
            out[i*n_ops+2:((i+1)*n_ops+2),:,:,:] = mat[i,-227:,:227,:]
            out[i*n_ops+3:((i+1)*n_ops+3),:,:,:] = mat[i,:227,-227:,:]
            out[i*n_ops+4:((i+1)*n_ops+4),:,:,:] = mat[i,-227:,-227:,:]
            
            # 5 mirrors
            for j in xrange(n_ops//2):
                out[i*n_ops+5+j:((i+1)*n_ops+5+j),:,:,:] = \
                out[i*n_ops+5+j:((i+1)*n_ops+5+j),:,:,::-1]           
        
    else:
        out = mat.repeat(n_ops, axis=0)
                
    return out

# Define model

In [29]:
from lasagne.layers import InputLayer, DenseLayer, Conv2DLayer, MaxPool2DLayer, DropoutLayer
from lasagne.layers import SliceLayer, ConcatLayer
import lasagne

def build_model(input_var=None):
    # Input layer
    ''' 
    out: b x 227 x 227 x 3
    '''
    lin = InputLayer(
        shape=(None, 227, 227, 3),
        input_var=input_var
        )

    # ConvPool1
    ''' 
    out: b x 96 x 27 x 27 
    out.W: 96 x 3 x 11 x 11
    '''
    """ input was b01c, need to be bc01"""
    l1 = Conv2DLayer(
        lasagne.layers.dimshuffle(lin, (0,3,1,2)),
        num_filters=96, filter_size=11, stride=4,
        W = Ws['W_0'], b = bs['b_0'],
        nonlinearity=lasagne.nonlinearities.rectify
        )
    l1 = MaxPool2DLayer(
        l1, pool_size=3, stride=2
        )

    # ConvPool2: 2 groups
    ''' 
    out: b x 256 x 13 x 13
    out.W0/1: 128 x 48 x 5 x 5
    '''
    l1_0 = SliceLayer(l1, indices=slice(None,48), axis=1)
    l2_0 = Conv2DLayer(
        l1_0, num_filters=128, filter_size=5, stride=1, pad=2,
        W = Ws['W0_1'], b = bs['b0_1'],
        nonlinearity=lasagne.nonlinearities.rectify
    )
    l2_0p = MaxPool2DLayer(
        l2_0, pool_size=3, stride=2
    )

    l1_1 = SliceLayer(l1, indices=slice(48, None), axis=1)
    l2_1 = Conv2DLayer(
        l1_1, num_filters=128, filter_size=5, stride=1, pad=2,
        W = Ws['W1_1'], b = bs['b1_1'],
        nonlinearity=lasagne.nonlinearities.rectify
    )
    l2_1p = MaxPool2DLayer(
        l2_1, pool_size=3, stride=2
    )

    l2 = ConcatLayer([l2_0p,l2_1p], axis=1)

    # Conv3
    ''' 
    out: b x 384 x 13 x 13
    out.W: 384 x 256 x 3 x 3
    '''
    l3 = Conv2DLayer(
        l2, num_filters=384, filter_size=3, stride=1, pad='same',
        W = Ws['W_2'], b = bs['b_2'],
        nonlinearity=lasagne.nonlinearities.rectify
    )

    # Conv4: 2 groups
    ''' 
    out: b x 384 x 13 x 13
    out.W0/1: 192 x 192 x 3 x 3
    '''
    l3_0 = SliceLayer(l3, indices=slice(None,192), axis=1)
    l4_0 = Conv2DLayer(
        l3_0, num_filters=192, filter_size=3, stride=1, pad='same',
        W = Ws['W0_3'], b = bs['b0_3'],
        nonlinearity=lasagne.nonlinearities.rectify
    )

    l3_1 = SliceLayer(l3, indices=slice(192, None), axis=1)
    l4_1 = Conv2DLayer(
        l3_1, num_filters=192, filter_size=3, stride=1, pad='same',
        W = Ws['W1_3'], b = bs['b1_3'],
        nonlinearity=lasagne.nonlinearities.rectify
    )

    # ConvPool5: 2 groups
    ''' 
    out: b x 256 x 6 x 6
    out.W0/1: 128 x 192 x 3 x 3
    '''
    l5_0 = Conv2DLayer(
        l4_0, num_filters=128, filter_size=3, stride=1, pad='same',
        W = Ws['W0_4'], b = bs['b0_4'],
        nonlinearity=lasagne.nonlinearities.rectify
    )
    l5_0p = MaxPool2DLayer(
        l5_0, pool_size=3, stride=2
    )

    l5_1 = Conv2DLayer(
        l4_1, num_filters=128, filter_size=3, stride=1, pad='same',
        W = Ws['W1_4'], b = bs['b1_4'],
        nonlinearity=lasagne.nonlinearities.rectify
    )
    l5_1p = MaxPool2DLayer(
        l5_1, pool_size=3, stride=2
    )

    l5 = ConcatLayer([l5_0p,l5_1p], axis=1)

    # FC6
    ''' 
    out: b x 4096 (x 1 x 1)
    out.W: 9216 x 4096
    '''
    l6 = DenseLayer(
        lasagne.layers.dropout(l5, p=.5),
        num_units=4096,
        W = Ws['W_5'], b = bs['b_5'],
        nonlinearity=lasagne.nonlinearities.rectify
    )

    # FC7
    ''' 
    out: b x 4096 (x 1 x 1)
    out.W: 4096 x 4096
    '''
    l7 = DenseLayer(
        lasagne.layers.dropout(l6, p=.5),
        num_units=4096,
        W = Ws['W_6'], b = bs['b_6'],
        nonlinearity=lasagne.nonlinearities.rectify
    )

    # FC8: replace last layer in AlexNet
    ''' 
    out: b x 22
    out.W: 4096 x 22
    '''
    l8 = DenseLayer(
        l7, num_units=22,     
        nonlinearity=lasagne.nonlinearities.softmax
    )
    return l8

In [30]:
net0 = build_model()

## Compiling Theano expressions


### Batch Iterator

In [13]:
# ############################# Batch iterator ###############################

"""Subtract IMAGE MEAN"""

def iterate_minibatches(inputs, targets, batchsize, shuffle=False):
    assert len(inputs) == len(targets), "Input and target size mismatch"
    if shuffle:
        indices = np.arange(len(inputs))
        np.random.shuffle(indices)
    for start_idx in range(0, len(inputs) - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], targets[excerpt]

In [34]:
# Prepare Theano variables for inputs and targets
input_var = T.tensor4('inputs')
target_var = T.imatrix('targets')

print "building model... "
net0 = build_model(input_var)

print "compiling functions... "
# Build loss function
prediction = lasagne.layers.get_output(net0)
loss = lasagne.objectives.categorical_crossentropy(prediction,
                                                   target_var)
loss = loss.mean()

# Create update expression for training
# using RMSprop
params = lasagne.layers.get_all_params(net0, 
                                       trainable=True)
updates = lasagne.updates.rmsprop(loss, params, 
                                  learning_rate=1.0, rho=0.9, epsilon=1e-06)
train_fn = theano.function([input_var, target_var], loss,
                           updates=updates)

## Building loss evaluation for validation set
va_prediction = lasagne.layers.get_output(net0, 
                                          deterministic=True)
va_loss = lasagne.objectives.categorical_crossentropy(va_prediction,
                                                      target_var)
va_loss = va_loss.mean()

va_fn = theano.function([input_var, target_var], va_loss)

building model... 
compiling functions... 


In [38]:
f = file('va_fn.pkl', 'wb')
cPickle.dump(va_fn, f, protocol=cPickle.HIGHEST_PROTOCOL)
f.close()

In [41]:
""" remember to save after every X epoch(s)"""
import time

def main(num_epochs=500, batchsize=96, toy=True, 
         Xtr=Xtr, Ytr=Ytr, Xva=Xva, Yva=Yva):
       
    # Sanity check: try to overfit a tiny (20 instances) subset of the data
    if toy: 
        np.random.RandomState(11)
        idx = np.random.randint(0,Xtr.shape[0]/10,20) * 10
        Xtr = Xtr[idx,:,:,:]
        Ytr = Ytr[idx,:]
        batchsize = 20

    print "Starting training with batchsize of %d ..." %(batchsize)    
    for epoch in range(num_epochs):
        # In each epoch, we do a full pass over the training data:
        train_err = 0
        train_batches = 0
        start_time = time.time()
        for batch in iterate_minibatches(Xtr, Ytr, batchsize, shuffle=True):
            inputs, targets = batch
            train_err += train_fn(inputs, targets)
            train_batches += 1

        # And a full pass over the validation data:
        if not toy:
            va_err = 0
            va_batches = 0
            for batch in iterate_minibatches(Xva, Yva, batchsize, shuffle=False):
                inputs, targets = batch
                err = va_fn(inputs, targets)
                va_err += err
                va_batches += 1

        # Then we print the results for this epoch:
        print("Epoch {} of {} took {:.3f}s".format(
            epoch + 1, num_epochs, time.time() - start_time))
        print("  training loss:\t\t{:.6f}".format(train_err / train_batches))
        if not toy:
            print("  validation loss:\t\t{:.6f}".format(va_err / va_batches))

            # Save the model 
            if epoch%2==0: 
                np.savez('Data/MSRCv2/model_'+str(epoch)+'.npz', lasagne.layers.get_all_param_values(net0))
        
        if toy and np.allclose(train_err,0):
            print "Error for toy problem is 0. Training finished"
            break


In [43]:
main()

Starting training with batchsize of 20 ...


KeyboardInterrupt: 

In [71]:
num_epochs=500
batchsize=20
toy=True

# Sanity check: try to overfit a tiny (20 instances) subset of the data
np.random.RandomState(11)
idx = np.random.randint(0,Xtr.shape[0]/10,20) * 10
Xtr_toy = Xtr[idx,:,:,:]
Ytr_toy = Ytr[idx,:]

In [73]:
Xtr_toy.dtype

dtype('uint8')

In [80]:
if np.any([isinstance(inputvar.op, T.Elemwise) for inputvar in train_fn.maker.fgraph.toposort()]):    print 'Used the cpu'
else:
    print 'Used the gpu'

Used the cpu


In [88]:

print "Starting training with batchsize of %d ..." %(batchsize)    
for epoch in range(num_epochs):
    # In each epoch, we do a full pass over the training data
    train_err = 0
    train_batches = 0
    start_time = time.time()


    train_err += va_fn(Xtr_toy, Ytr_toy)


    # Then we print the results for this epoch:
    print("Epoch {} of {} took {:.3f}s".format(
        epoch + 1, num_epochs, time.time() - start_time))
    print("  training loss:\t\t{:.6f}".format(train_err))

    if np.allclose(train_err,0):
        print "Error for toy problem is 0. Training finished"
        break

Starting training with batchsize of 20 ...


RuntimeError: Cuda error 'an illegal instruction was encountered' while copying %lli data element to device memory
Apply node that caused the error: GpuFromHost(inputs)
Toposort index: 23
Inputs types: [TensorType(float32, 4D)]
Inputs shapes: [(20, 227, 227, 3)]
Inputs strides: [(618348, 2724, 12, 4)]
Inputs values: ['not shown']
Outputs clients: [[GpuDimShuffle{0,3,1,2}(GpuFromHost.0)]]

HINT: Re-running with most Theano optimization disabled could give you a back-trace of when this node was created. This can be done with by setting the Theano flag 'optimizer=fast_compile'. If that does not work, Theano optimizations can be disabled with 'optimizer=None'.
HINT: Use the Theano flag 'exception_verbosity=high' for a debugprint and storage map footprint of this apply node.